# CS522 - Final Project
The objective of the final project is to integrate various machine learning techniques to achieve the best performance. Final project is a group effort. Each group can have 4-5 members. You are required to apply ALL techniques learned in this semester.

## Schedule
- (5) Milestone 1 (Due 11/4): Group Formation and Topic selection. Submit through Canvas. Approval and comments will be returned in one day. The same topic cannot be chosen by more than 1 group. The topic follows the first-come first-served rule. So pick a topic as soon as possible.
- (5) Milestone 2 - Literature Survey (Due 11/11): Background study including references and state-of-the-art performance on the dataset (2-page report need to be submitted).
- (5) Milestone 3 - Prototype 1 (Due 11/18): Prototype, preliminary results and task allocation among group members. Apply at least one learned technique successfully for each component in the pipeline on the chosen dataset and submit a 1-page report.
- (5) Milestone 4 - Prototype 2 (Due 12/02): Implement at least two solutions to each component of the pipeline. Determine what metrics to use. Provide performance evaluation results.
- (100) Final presentation (Due 12/08)(Presentation slides due the midnight before the presentation on 12/9. Submit through Canvas)
- (80) Final report (Due 12/10). Submit through Canvas.

## Potential Topics
Each group can choose one topic from the following sources. All selection needs to be approved by instructor.
- KDD-Cup 1997-2009
- Kaggle Competitions
- Other topics: You can select a topic yourself from other resources.

## Requirement
General steps involved in a machine learning problem include
- Data collection (raw data)
- Feature extraction (how to extract features from the raw data)
- Feature selection (dimensionality reduction - Fisher's linear discriminant or PCA)
- Classification/Regression methods need to be included
    - Supervised learning and Unsupervised learning
    - Baysian approaches and non-Baysian approaches
    - Parametric and Non-parametric density estimation in supervised learning
    - Fusion
- Performance evaluation
- Feedback system

You are required to evaluate the effect of various aspects of the classification/regression process, including but not limited to
- the effect of assuming the data is Gaussian-distributed
- the effect of assuming parametric pdf vs. non-parametric pdf
- the effect of using different prior probability ratio
- the effect of using different distance
- the effect of knowing the class label
- the effect of dimension of the feature space (e.g., changed through dimensionality reduction)
- the effect of fusion

To be more specific, you need to at least go through the following steps:
- Data normalization
- Dimensionality reduction
    - Classification/Regression with the following
    - MPP (case 1, 2, and 3)
    - kNN with different k's
    - BPNN
    - Decision tree
    - SVM
    - Clustering (kmeans, wta)
- Classifier fusion
- Evaluation (use n-fold cross validation to generate confusion matrix and ROC curve if applicable).

# 1. Data collection (raw data) - Face Mask 12k Images Dataset
- URL: [https://www.kaggle.com/ashishjangra27/face-mask-12k-images-dataset](https://www.kaggle.com/ashishjangra27/face-mask-12k-images-dataset)

This dataset is used for Face Mask Detection Classification with images. The dataset consists of almost 12K images which are almost 328.92MB in size.

"All the images with the face mask (~6K) are scrapped from google search and all the images without the face mask are preprocessed from the CelebFace dataset created by Jessica Li (https://www.kaggle.com/jessicali9530). Thank you so much Jessica for providing a wonderful dataset to the community."


In [2]:
%load_ext autoreload
%autoreload 2

# IMPORT LIBRARIES

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# IMPORT OUR OWN FILES

from Dataset import *
from PreProcessing import *
from convolution import *

In [4]:
# MISCELLANEOUS ROUTINES

# make directory to save generated images to
os.makedirs('img', exist_ok=True)

## 1.1. Read In Data
Throughout this project:
- 0 corresponds to not wearing a mask
- 1 corresponds to wearing a mask

In [5]:
dataset = Dataset(dataset_dir=os.path.join("datasets", "face_mask"))
dataset.load_all()

print(dataset)

# Dataset Init: datasets/face_mask
	- Loading Datasets...
	=> TRAIN
		- With Mask: 5000 (50.0%)
		- No Mask: 5000 (50.0%)
		- Total: 10000
	=> VALIDATION
		- With Mask: 400 (50.0%)
		- No Mask: 400 (50.0%)
		- Total: 800
	=> TEST
		- With Mask: 483 (48.69%)
		- No Mask: 509 (51.31%)
		- Total: 992
# Loading Train Set...


100%|██████████| 10000/10000 [00:07<00:00, 1351.11it/s]


# Loading Validation Set...


100%|██████████| 800/800 [00:00<00:00, 1409.44it/s]


# Loading Test Set...


100%|██████████| 992/992 [00:00<00:00, 1421.34it/s]

	- Datasets Loaded!
# Dataset Info: datasets/face_mask
	- Train: 10000 (84.8%)
	- Validation: 800 (6.78%)
	- Test: 992 (8.41%)
	- Total: 11792


## 1.2. Normalizing dataset
Need to standardize the size of the images.
It is conventional to resize all of the images to the smallest image in the dataset.
https://datascience.stackexchange.com/questions/40462/how-to-prepare-the-varied-size-input-in-cnn-prediction



In [6]:
def t_resize(X):
    # resize training data
    return PreProcessing.resize_images(X, size=(128, 128, 3))


dataset.transform(t_resize)


def t_grayscale(X):
    return PreProcessing.dataset2grayscale(X)


dataset_gray = dataset.copy_transform(t_grayscale)


def t_scale(X):
    return PreProcessing.min_max_scale_dataset(X)


dataset.transform(t_scale)
dataset_gray.transform(t_scale)

# Performing Feature Set Transformation: t_resize
	- 1/3


0it [00:00, ?it/s]


AttributeError: 'cupy._core.core.ndarray' object has no attribute '__array_interface__'

## 1.3. Peek at Dataset
In order to get a feel for the data, lets look at some of the images

In [ ]:
def plot_img_grid(img, label, size, pos):
    """
    given the image, its label, the size of the grid, and its position on the grid,
    create a grid of images displaying images in the dataset
    """
    ax = plt.subplot2grid(size, pos)
    ax.tick_params(left=False, right=False, labelleft=False,
                   labelbottom=False, bottom=False)
    ax.title.set_text(label)
    ax.imshow(img, cmap='gray')


fig = plt.figure(figsize=(8, 6))
fig.suptitle('Quick peek at the data')
plot_img_grid(dataset_gray.train.X[0], 'No Mask', (2, 3), (0, 0))
plot_img_grid(dataset_gray.train.X[1000], 'No Mask', (2, 3), (0, 1))
plot_img_grid(dataset_gray.train.X[2000], 'No Mask', (2, 3), (0, 2))
plot_img_grid(dataset_gray.train.X[7000], 'Mask', (2, 3), (1, 0))
plot_img_grid(dataset_gray.train.X[8001], 'Mask', (2, 3), (1, 1))
plot_img_grid(dataset_gray.train.X[9004], 'Mask', (2, 3), (1, 2))
plt.tight_layout()
plt.savefig('./img/faces.png', dpi=500)
plt.show()


# 2. Feature extraction
Objective: how to extract features from the raw data

# 2.1. Convolutional Neural Network (CNN)

![CNN Hierarchy](img/cnn_hierarchy.png)

Build Layers and Sequential Model for the convolution


In [ ]:
def get_cnn_rgb(flatten: bool):
    kernel = np.array([[[1, -1, 1], [-1, 8, -1], [1, -1, 1]], [[1, -1, 1], [-1, 8, -1], [1, -1, 1]],
                       [[1, -1, 1], [-1, 8, -1], [1, -1, 1]]])  # 3d kernel needed for 3d image

    model = Sequential()
    model.add(Conv(kernel=kernel, name="input_layer"))
    model.add(Activation('relu'))
    model.add(Pooling2D(pool_size=2, stride=2, padding=0, mode='avg', name="pooling1"))
    model.add(Conv(kernel=kernel, name="layer1"))
    model.add(Activation('relu'))
    model.add(Pooling2D(pool_size=2, stride=2, padding=0, mode='avg', name="pooling1"))
    if flatten:
        model.add(Flatten())
    model.summary()
    return model


test_image = dataset.train.X[0]
x = get_cnn_rgb(flatten=False).feedforward(test_image)

# display original image
plt.imshow(test_image)
plt.show()

# display convolved image
plt.imshow(x)
plt.show()

In [ ]:

def get_cnn_gray(flatten: bool):
    # convolution kernel
    # kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]]) # sharper images
    kernel = np.array([[0, -1, 0], [-1, 6, -1], [0, -1, 0]])  # sharper images
    # kernel = np.array([[-1, -1, -1], [-1, 0, -1], [-1, -1, -1]])
    # kernel = np.array([[[-1, 0, -1], [-1, 0, -1], [-1, 0, -1]],
    #                    [[-1, 0, -1], [-1, 0, -1], [-1, 0, -1]],
    #                    [[-1, 0, -1], [-1, 0, -1], [-1, 0, -1]]])
    # kernel = np.array([[0,1,2,1,0], [1,4,8,4,1], [2,8,16,8,2], [1,4,8,4,1],[0,1,2,1,0]]) # smoothing
    # kernel = np.array([[0, 1, 0], [1, 4, 1], [0, 1, 0]])  # gaussian blur

    # build model
    model = Sequential()
    model.add(Conv(kernel=kernel, name="input_layer"))
    model.add(Activation('relu'))
    model.add(Pooling2D(mode='avg'))
    model.add(Conv(kernel=kernel, name="input_layer"))
    model.add(Activation('relu'))
    model.add(Conv(kernel=kernel, name="input_layer"))
    if (flatten):
        model.add(Flatten())
    # model.add(Activation('sigmoid'))
    model.summary()
    return model


test_image_gray = dataset_gray.train.X[7345]
# feed image through model
x = get_cnn_gray(flatten=False).feedforward(test_image_gray)

# display original image
plt.imshow(test_image_gray, cmap='gray')
plt.show()

# display convoluted image
plt.imshow(x, cmap='gray')
plt.show()


In [ ]:
from tqdm import tqdm
model_cnn_rgb = get_cnn_rgb(flatten=True)
model_cnn_gray = get_cnn_gray(flatten=True)

def t_conv_rgb(X):
    conv = np.array([model_cnn_rgb.feedforward(x) for x in tqdm(X)])
    return conv

def t_conv_gray(X):
    conv = np.array([model_cnn_gray.feedforward(x) for x in tqdm(X)])
    return conv

dataset.transform(t_conv_rgb)
dataset_gray.transform(t_conv_gray)

In [ ]:
# Save transformed dataset objects to file
dirpath = os.path.join('datasets', 'face_mask_pickled')

dataset.save_gzip(dir_path=dirpath, file_name="dataset_conv.pkl.gzip", overwrite=True)
dataset_gray.save_gzip(dir_path=dirpath, file_name="dataset_gray_conv.pkl.gzip", overwrite=True)

# 3. Feature Selection
Dimensionality Reduction - Fisher's linear discriminant or PCA

In [ ]:
print("# Train")
X_train_norm = np.array(dataset.train.X)
X_train_gray_norm = np.array(dataset_gray.train.X)
print(f"* X_train_norm shape {X_train_norm.shape}")
print(f"* X_train_gray_norm shape {X_train_gray_norm.shape}")
print(f"* y_train shape {dataset.train.y.shape}")
print()

print("# Validation")
X_validation_norm = np.array(dataset.validation.X)
X_validation_gray_norm = np.array(dataset_gray.validation.X)
print(f"* X_validation_norm shape {X_validation_norm.shape}")
print(f"* X_validation_gray_norm shape {X_validation_gray_norm.shape}")
print(f"* y_validation shape {dataset.validation.y.shape}")
print()

print("# Test")
X_test_norm = np.array(dataset.test.X)
X_test_gray_norm = np.array(dataset_gray.test.X)
print(f"* X_test_norm shape {X_test_norm.shape}")
print(f"* X_test_gray_norm shape {X_test_gray_norm.shape}")
print(f"* y_test shape {dataset.test.y.shape}")
print()

# Flatten array (without CNN for now until we run and save all images)

In [ ]:
# this might take a while
# save the output of the convoluted training examples in case we want to use them later
"""
dir_path = os.path.join('datasets', 'face_mask_pickled')
os.makedirs(dir_path, exist_ok=True)


def get_conv_dataset(dataset, fn):
    if os.path.exists(fn):
        with gzip.open(fn, 'rb') as f:
            conv = pickle.load(f)
    else:
        conv = np.array([model.feedforward(x) for x in dataset])
        with gzip.open(fn, 'wb') as f:
            pickle.dump(conv, f)

    return conv


X_train_gray_conv = get_conv_dataset(X_train_gray_norm, os.path.join(dir_path, 'X_train_gray_conv.pkl.gzip'))
X_validation_gray_conv = get_conv_dataset(X_validation_gray_norm,
                                          os.path.join(dir_path, 'X_validation_gray_conv.pkl.gzip'))
X_test_gray_conv = get_conv_dataset(X_test_gray_norm, os.path.join(dir_path, 'X_test_gray_conv.pkl.gzip'))
"""

# 4. Classification/Regression methods need to be included
- Supervised learning and Unsupervised learning
- Baysian approaches and non-Baysian approaches
- Parametric and Non-parametric density estimation in supervised learning
- Fusion

Classification performed with:
1. MPP (case 1, 2, and 3)
2. kNN with different k's
3. BPNN
4. Decision tree
5. SVM
6. Clustering (kmeans, wta)

You are required to evaluate the effect of various aspects of the classification/regression process, including but not limited to:
- the effect of assuming the data is Gaussian-distributed
- the effect of assuming parametric pdf vs. non-parametric pdf
- the effect of using different prior probability ratio
- the effect of using different distance
- the effect of knowing the class label
- the effect of dimension of the feature space (e.g., changed through dimensionality reduction)
- the effect of fusion

## 4.1. MPP (case 1, 2, and 3)

In [ ]:
from mpp import MPPCase1
from evaluation import *

case_1 = MPPCase1([0.5, 0.5])
case_1.fit(X_train_gray_conv, y_train)
y_pred = case_1.predict(X_test_gray_conv)

cm = get_confusion_matrix(y_test, y_pred, np.unique(y_test))
plot_confusion_matrix(cm, np.unique(y_test), 'Case 1 Confusion Matrix')

## 4.2. kNN with different k's

## 4.3. BPNN

## 4.4. Decision Tree

## 4.5. SVM

## 4.6. Clustering (KMeans and WTA)

In [ ]:

from data_functions import get_dataset_fn, load_dataset, resize_dataset

# plt.imshow(X_train[0])
# plt.show()

# plt.imshow(kmeans_image_compress(X_train[0], 3))
# plt.show()
dataset_dir = os.path.join("datasets", "face_mask")
size = (128, 128, 3)
X_train_resized = resize_dataset(load_dataset(get_dataset_fn(dataset_dir, "train")[0][:10]), size)


# 5. Performance evaluation
Note: Use n-fold cross validation to generate confusion matrix and ROC curve if applicable.

# 6. Feedback system
Maybe